In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [3]:
# Function to extract Product Title
def get_title(soup):
    try:
        title = soup.find("span", attrs={"id":'productTitle'})
        title_value = title.text
        title_string = title_value.strip()
    except AttributeError:
        title_string = ""
    return title_string

# Function to extract Product Price
def get_price(soup):
    try:
        price = soup.find("span", attrs={'id':'priceblock_ourprice'}).string.strip()
    except AttributeError:
        try:
            price = soup.find("span", attrs={'id':'priceblock_dealprice'}).string.strip()
        except:
            price = ""
    return price

# Function to extract Product Rating
def get_rating(soup):
    try:
        rating = soup.find("i", attrs={'class':'a-icon a-icon-star a-star-4-5'}).string.strip()
    except AttributeError:
        try:
            rating = soup.find("span", attrs={'class':'a-icon-alt'}).string.strip()
        except:
            rating = ""
    return rating

# Function to extract Number of User Reviews
def get_review_count(soup):
    try:
        review_count = soup.find("span", attrs={'id':'acrCustomerReviewText'}).string.strip()
    except AttributeError:
        review_count = ""
    return review_count

# Function to extract Availability Status
def get_availability(soup):
    try:
        available = soup.find("div", attrs={'id':'availability'})
        available = available.find("span").string.strip()
    except AttributeError:
        available = "Not Available"
    return available


In [5]:
if __name__ == '__main__':
    # Add your user agent 
    HEADERS = ({'User-Agent':"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36", 'Accept-Language': 'en-US, en;q=0.5'})

    # The webpage URL
    URL = "https://www.amazon.com/s?k=playstation+4&ref=nb_sb_noss_2"

    # HTTP Request
    webpage = requests.get(URL, headers=HEADERS)

    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")

    # Fetch links as List of Tag Objects
    links = soup.find_all("a", attrs={'class':'a-link-normal s-no-outline'})

    # Store the links
    links_list = []

    # Loop for extracting links from Tag Objects
    for link in links:
        links_list.append(link.get('href'))

    # Dictionary to store the data
    d = {"title":[], "price":[], "rating":[], "reviews":[],"availability":[]}

    # Loop for extracting product details from each link
    for link in links_list:
        # Construct the full URL
        product_url = "https://www.amazon.com" + link
        new_webpage = requests.get(product_url, headers=HEADERS)
        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        # Function calls to display all necessary product information
        d['title'].append(get_title(new_soup))
        d['price'].append(get_price(new_soup))
        d['rating'].append(get_rating(new_soup))
        d['reviews'].append(get_review_count(new_soup))
        d['availability'].append(get_availability(new_soup))

    # Creating a DataFrame
    amazon_df = pd.DataFrame.from_dict(d)

    # Replace empty strings with NaN
    amazon_df['title'] = amazon_df['title'].replace('', np.nan)

    # Drop rows where 'title' is NaN
    amazon_df = amazon_df.dropna(subset=['title'])

    # Saving the data to a CSV file
    amazon_df.to_csv("amazon_data.csv", header=True, index=False)

    # Display the DataFrame
    print(amazon_df)

                                                title price  \
1   PlayStation®4 Console – Call of Duty® Modern W...         
2   Playstation Sony 4, 500GB Slim System [CUH-221...         
3   Wireless Controller Dual Vibration Game Joysti...         
4     $250 PlayStation Store Gift Card [Digital Code]         
5   PS4 Controller Charger Dock Station, OIVO 1.8H...         
6   Wireless Controller for PS4, Custom Design V2 ...         
7           Grand Theft Auto V: Premium Edition (PS4)         
8   Playstation 4 Slim 2TB SSD Console with Dualsh...         
9   FASIGO PS4 Controller 2 Pack, Wireless PS4 Con...         
10  PS4 Controller Wireless 2 Pack, Compatible wit...         
11        PowerA USB Charging Cable for PlayStation 4         
12  KDYGPDCT 2 Pack Wireless Controller for PS4/Pr...         
13             Sony Playstation PS4 1TB Black Console         
14                     PlayStation 4 Slim 1TB Console         
15  PlayStation 4 Slim 1TB Limited Edition Console...  

In [6]:
amazon_df

,title,price,rating,reviews,availability
1,PlayStation®4 Console – Call of Duty® Modern W...,,4.5 out of 5 stars,99 ratings,In Stock
2,"Playstation Sony 4, 500GB Slim System [CUH-221...",,4.5 out of 5 stars,353 ratings,In Stock
3,Wireless Controller Dual Vibration Game Joysti...,,4.0 out of 5 stars,"2,217 ratings",In Stock
4,$250 PlayStation Store Gift Card [Digital Code],,4.7 out of 5 stars,"250,741 ratings",Available now
5,"PS4 Controller Charger Dock Station, OIVO 1.8H...",,4.7 out of 5 stars,"38,850 ratings",In Stock
6,"Wireless Controller for PS4, Custom Design V2 ...",,4.2 out of 5 stars,"2,710 ratings",In Stock
7,Grand Theft Auto V: Premium Edition (PS4),,4.7 out of 5 stars,"12,224 ratings",In Stock
8,Playstation 4 Slim 2TB SSD Console with Dualsh...,,Previous page of related Sponsored Products,,Not Available
9,"FASIGO PS4 Controller 2 Pack, Wireless PS4 Con...",,4.3 out of 5 stars,115 ratings,In Stock
10,"PS4 Controller Wireless 2 Pack, Compatible wit...",,4.2 out of 5 stars,72 ratings,In Stock
